# MTG Database Interface
Scryfall API -> google cloud mongoDB -> Streamlit App

## 1. Data Collection

In [24]:
from bs4 import BeautifulSoup as bs
import requests

#get html code for scryfall bulk download page
bulk_url = requests.get('https://scryfall.com/docs/api/bulk-data')

#convert html to soup
soup = bs(bulk_url.content, 'html.parser')

In [27]:
#find URL for bulk download, two items after the table row title, "Default Cards"
link = soup.find(text = "Default Cards").next.next
#print(link)

<td><a href="https://c2.scryfall.com/file/scryfall-bulk/default-cards/default-cards-20220928210728.json">Download</a></td>


In [29]:
import re
#use regex to extract text between quotations. convert soup to string
url = re.findall(r'"([^"]*)"', str(link))
#print(url)

['https://c2.scryfall.com/file/scryfall-bulk/default-cards/default-cards-20220928210728.json']


In [31]:
#https://c2.scryfall.com/file/scryfall-bulk/default-cards/default-cards-20220928210728.json
#URL for bulk download of all english MTG card objects on scryfall
#import requests

bulk = requests.get(url[0])
#print(bulk.status_code) #200 = ok

200


In [6]:
type(bulk.content)

bytes

In [7]:
mtg_lib = bulk.json()

/home/jtaz/anaconda3/lib/python3.9/json/decoder.py:353: RuntimeWarning: coroutine 'FoundationObject.__init__.<locals>.main' was never awaited
  obj, end = self.scan_once(s, idx)


In [8]:
print(type(mtg_lib))

<class 'list'>


In [17]:
print(len(mtg_lib))
print(mtg_lib[0])

73262
{'object': 'card', 'id': '0000579f-7b35-4ed3-b44c-db2a538066fe', 'oracle_id': '44623693-51d6-49ad-8cd7-140505caf02f', 'multiverse_ids': [109722], 'mtgo_id': 25527, 'mtgo_foil_id': 25528, 'tcgplayer_id': 14240, 'cardmarket_id': 13850, 'name': 'Fury Sliver', 'lang': 'en', 'released_at': '2006-10-06', 'uri': 'https://api.scryfall.com/cards/0000579f-7b35-4ed3-b44c-db2a538066fe', 'scryfall_uri': 'https://scryfall.com/card/tsp/157/fury-sliver?utm_source=api', 'layout': 'normal', 'highres_image': True, 'image_status': 'highres_scan', 'image_uris': {'small': 'https://c1.scryfall.com/file/scryfall-cards/small/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979', 'normal': 'https://c1.scryfall.com/file/scryfall-cards/normal/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979', 'large': 'https://c1.scryfall.com/file/scryfall-cards/large/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979', 'png': 'https://c1.scryfall.com/file/scryfall-cards/png/front/0/0/000057

## 2. Import JSON file to MongoDB

In [15]:
from pymongo import MongoClient

#create client instance
client = MongoClient()

# create cards database and assign to db
db = client.mtg
print('Databases: \t', client.list_database_names())

#create card collection in mtg database
cards = db.cards

In [16]:
#add cards json from request to cards collection
cards.insert_many(mtg_lib)

#check for number of cards added, should be 73,262
cards.estimated_document_count()

73262

In [19]:
# test db inputs with query for card "Fury Sliver", 2 entries returned for 2 versions of the card
list(db.cards.find({'name':'Fury Sliver'}))

[{'_id': ObjectId('6334e24dcdf7a68bf95a8355'),
  'object': 'card',
  'id': '0000579f-7b35-4ed3-b44c-db2a538066fe',
  'oracle_id': '44623693-51d6-49ad-8cd7-140505caf02f',
  'multiverse_ids': [109722],
  'mtgo_id': 25527,
  'mtgo_foil_id': 25528,
  'tcgplayer_id': 14240,
  'cardmarket_id': 13850,
  'name': 'Fury Sliver',
  'lang': 'en',
  'released_at': '2006-10-06',
  'uri': 'https://api.scryfall.com/cards/0000579f-7b35-4ed3-b44c-db2a538066fe',
  'scryfall_uri': 'https://scryfall.com/card/tsp/157/fury-sliver?utm_source=api',
  'layout': 'normal',
  'highres_image': True,
  'image_status': 'highres_scan',
  'image_uris': {'small': 'https://c1.scryfall.com/file/scryfall-cards/small/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979',
   'normal': 'https://c1.scryfall.com/file/scryfall-cards/normal/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979',
   'large': 'https://c1.scryfall.com/file/scryfall-cards/large/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?15628